In [1]:
# 统计各个粒度下的准确率
import pandas as pd
import json
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
source = pd.read_csv('/home/linzhisheng/esg/QA/new/report_all_eng.csv')
# source = source.drop(columns=['text'])
# print(source)
result = pd.read_csv('/home/linzhisheng/esg/QA/new/deberta_firuge_1000.csv')
result2 = pd.read_csv('/home/linzhisheng/esg/QA/new/deberta_firuge_1000_new_path.csv')
result3 = pd.read_csv('/home/linzhisheng/esg/QA/new/deberta_firuge_1000_table.csv')
# result = pd.read_csv('/home/linzhisheng/esg/QA/new/figure_1000_prob_all_new.csv')
# result2 = pd.read_csv('/home/linzhisheng/esg/QA/new/figure_1000_prob_all_new_path.csv')
# print(result)
# print(result2)
import json
count = 0
valid_count = 0
topn = 10
path_map = {}
hit_map = {}
res_map = []
url_map = {}
wrong_count = 0
no_exist_count = 0
small_number_count = 0
filter = ['NOxEmissions','ElectricityPurchased', 'TotalInjuryRateEmployees','CO2EquivalentsEmissionIndirectScope3','EnergyUseTotal']
# filter = ['CO2EquivalentsEmissionIndirectScope3']
# filter = ['EnergyUseTotal']
# filter = ['NOxEmissions']
for i,row in source.iterrows():
    path = row['path']
    target = row['value']
    text = row['text']
    idx = row['Unnamed: 0.1']
    number = locale.atof(target)
    # if path not in filter:
    #     continue
    if text.find(target) < 0:
        no_exist_count = no_exist_count + 1
        continue
    # if number < 10 and target.isdigit():
    #     # print(target)
    #     small_number_count = small_number_count + 1
    #     continue

        # print((target, row['url']))
    valid_count = valid_count + 1    
    if path in path_map:
        path_map[path] = path_map[path] + 1
    else:
        path_map[path] = 1
     
     
     

    answer_set = result.iloc[i]['answer_set']
    limit = 0
    find = 0
    for ii in answer_set.split("{\'answer\': "):
        limit = limit + 1
        if limit >= topn + 2:
            break
        ans = ii[1:ii.find('logit_score')-4]
        # if ans.find(target) >=0 :
        ans_no_space = ans.replace(' ','')
        
        if ans_no_space != ans and ans_no_space == target:
            count = count + 1
            find = 1
            if path in hit_map:
                hit_map[path] = hit_map[path] + 1
            else:
                hit_map[path] = 1
            break
            # print(ans_no_space)
        elif target =='0' or ans == target:
            # print((count,target,ans))
            count = count + 1
            if path in hit_map:
                hit_map[path] = hit_map[path] + 1
            else:
                hit_map[path] = 1
            find = 1
            break
        elif ans.find(target) >=0:
            count = count + 1
            if path in hit_map:
                hit_map[path] = hit_map[path] + 1
            else:
                hit_map[path] = 1
            find = 1
            break
        # print((target,ans))
        
        
        
        
    limit = 0
    if find == 0:
        answer_set = result2.iloc[i]['answer_set']
        for ii in answer_set.split("{\'answer\': "):
            limit = limit + 1
            if limit >= topn + 2:
                break
            ans = ii[1:ii.find('logit_score')-4]
            # if ans.find(target) >=0 :
            ans_no_space = ans.replace(' ','')
            
            if ans_no_space != ans and ans_no_space == target:
                count = count + 1
                find = 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                break
                # print(ans_no_space)
            if target =='0' or ans == target:
                # print((count,target,ans))
                count = count + 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                find = 1
                break
            if ans.find(target) >=0:
                count = count + 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                find = 1
                break
            
    limit = 0
    if find == 0:
        answer_set = result3.iloc[i]['answer_set']
        for ii in answer_set.split("{\'answer\': "):
            limit = limit + 1
            if limit >= topn + 2:
                break
            ans = ii[1:ii.find('logit_score')-4]
            # if ans.find(target) >=0 :
            ans_no_space = ans.replace(' ','')
            
            if ans_no_space != ans and ans_no_space == target:
                count = count + 1
                find = 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                break
                # print(ans_no_space)
            if target =='0' or ans == target:
                # print((count,target,ans))
                count = count + 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                find = 1
                break
            if ans.find(target) >=0:
                count = count + 1
                if path in hit_map:
                    hit_map[path] = hit_map[path] + 1
                else:
                    hit_map[path] = 1
                find = 1
                break
            
    if find == 0:
        wrong_count = wrong_count + 1
        # print((idx, wrong_count, row['value'],row['path'],row['url'], answer_set[:100]))    
    # print(answer_set[0])
    
for key in path_map:
    if key not in hit_map:
        res_map.append({
            'path':key,
            'count':path_map[key],
            'score':0
        })
        # res_map[key] = (path_map[key],0)
    else:
        res_map.append({
            'path':key,
            'count':path_map[key],
            'score':hit_map[key]/path_map[key]
        })
        # res_map[key] = (path_map[key],hit_map[key]/path_map[key])
    
import heapq

# print(res_map)
res = heapq.nlargest(100, res_map, key=lambda s: s['count'])
print(json.dumps(res,indent=1))
# print(json.dumps(res_map, indent=1))
# print(json.dumps(hit_map, indent=2))
print('{}'.format(count/valid_count))
print((count,no_exist_count,wrong_count,valid_count))

# print(table_num)
# print('{}'.format(table_num/len(source)))

[
 {
  "path": "ElectricityPurchased",
  "count": 467,
  "score": 0.9314775160599572
 },
 {
  "path": "EnergyUseTotal",
  "count": 337,
  "score": 0.8397626112759644
 },
 {
  "path": "WasteRecycledTotal",
  "count": 240,
  "score": 0.9041666666666667
 },
 {
  "path": "HazardousWaste",
  "count": 200,
  "score": 0.815
 },
 {
  "path": "RenewableEnergyProduced",
  "count": 191,
  "score": 0.9214659685863874
 },
 {
  "path": "DonationsTotal",
  "count": 184,
  "score": 0.7771739130434783
 },
 {
  "path": "EnergyPurchasedDirect",
  "count": 183,
  "score": 0.8415300546448088
 },
 {
  "path": "CO2EquivalentsEmissionDirectScope1",
  "count": 177,
  "score": 0.847457627118644
 },
 {
  "path": "CO2EquivalentsEmissionIndirectScope3",
  "count": 156,
  "score": 0.7692307692307693
 },
 {
  "path": "ElectricityProduced",
  "count": 146,
  "score": 0.8904109589041096
 },
 {
  "path": "NonHazardousWaste",
  "count": 127,
  "score": 0.7874015748031497
 },
 {
  "path": "WasteTotal",
  "count": 122,
  

In [13]:
import pandas as pd
import json
source = pd.read_csv('/home/linzhisheng/esg/QA/new/report_all_format2.csv')
print('read done')
for i,row in source.iterrows():
    path = row['path']
    target = row['value']
    url = row['url']
    if url.find('https://www.japfa.com/files/report/Japfa_Ltd_Sustainability_Report_2020.pdf') == 0:
        # print(row['text'])
        if row['text']:
            print(target)
            f = open('example.txt', 'w')
            f.write(row['text'])
            break



read done
951.3


In [1]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

train = pd.read_csv('/home/linzhisheng/esg/QA/new/report_all_eng.csv')
print(train['path'])
map = {}
counts = train['path'].value_counts()
for key, value in counts.iteritems():
    words = re.findall('[A-Z][^A-Z]*', key)
    # s_new = ''.join([s for s in words if len(s) == 1])
    # print(key, value)
    new_path = (' '.join(words)).lower()
    new_path = new_path.replace('c o2', 'co2')
    new_path = new_path.replace('n ox', 'nox')
    new_path = new_path.replace('s ox', 'sox')
    new_path = new_path.replace('e s g', 'esg')
    map[key] = new_path
    # print((' '.join(words)).lower())
for index, row in train.iterrows():
    # 获取行的索引值
    row_index = index
    
    # 获取行的值
    row_value = row
    
    # 修改行的值
    row_value['path'] = map[row_value['path']]
    
    # 将修改后的行值赋回数据框中对应的位置
    train.loc[row_index] = row_value
train.to_csv('/home/linzhisheng/esg/QA/new/report_all_eng_new.csv',index=False)

/home/linzhisheng/esg-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


0       ElectricityPurchased                      
1       NOxEmissions                              
2       CO2EquivalentsEmissionIndirectScope3      
3       ElectricityProduced                       
4       CarbonOffsetsCredits                      
5       EnergyProducedDirect                      
6       WaterDischarged                           
7       ElectricityPurchased                      
8       EnergyUseTotal                            
9       HazardousWaste                            
10      ElectricityPurchased                      
11      EnergyUseTotal                            
12      ElectricityPurchased                      
13      NonHazardousWaste                         
14      ElectricityPurchased                      
15      EnergyUseTotal                            
16      WasteRecycledTotal                        
17      HazardousWaste                            
18      ElectricityPurchased                      
19      WasteRecycledTotal     

In [8]:
import pandas as pd
test = pd.read_csv('/home/linzhisheng/esg/QA/new/test.csv')
def is_chinese(string):
    # print(string)
    count = 0
    string = str(string)
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            count = count + 1
            if count > 0:
                return True
    return False
test
test = test[test['text'].apply(lambda x: is_chinese(x) != True)]
test

,Unnamed: 0,value,text,type,path,url
0,7,10,"Electricity 9,582 GWh \nHeat 738 TJ \nFuels 9,...",ResourceUse,ElectricityPurchased,https://www.nipponsanso-hd.co.jp/Portals/0/ima...
1,9,5197.33,1.3.3\nEMISSIONS TO AIR\nMagyar Telekom pays a...,Emission,NOxEmissions,https://www.telekom.hu/static-tr/sw/file/susta...
2,22,"2,100",CO2e is the amount of CO2 and the amount of no...,Emission,CO2EquivalentsEmissionIndirectScope3,http://www.morganadvancedmaterials.com/media/4...
3,24,"335,610","Solar Energy\n(...)\nWe have approximately 1,0...",ResourceUse,ElectricityProduced,http://www.hindustanpetroleum.com/documents/pd...
4,40,354,"Table: Rovio's responsibility focus areas, per...",Emission,CarbonOffsetsCredits,https://investors.rovio.com/sites/rovio-ir-v2/...
...,...,...,...,...,...,...
4504,31460,2.5,Name of the performance\nindicator Scope of th...,Workforce,TotalInjuryRateEmployees,https://femsa.gcs-web.com/static-files/bf9eda0...
4505,31466,1,\n\n\nGeneral\nStandard\nDisclosure\nDisclosur...,Workforce,TradeUnionRepresentation,https://www.ivanhoemines.com/site/assets/files...
4506,31480,1.62,"Health, safety and environment (""HS&E"")\n........",Workforce,TotalInjuryRateTotal,https://www.parkland.ca/application/files/7115...
4507,31488,"400,000","In broad terms, GCOs financial social\nrespons...",Community,DonationsTotal,https://www.mineraldeposits.com.au/wp-content/...
